## Import libraries

In [1]:
import json
from src.utils import *

from langchain_core.prompts import PromptTemplate
from langchain_google_genai import GoogleGenerativeAI

from src.prompt import extract_keyword_prompt, important_inquiry_prompt
from dotenv import load_dotenv

In [2]:
load_dotenv()

True

## Extract

In this step, we will apply 2 methods to extract insightful data from customer's message:
- **Extracting keyword**: Use LLM to distil important keywords in messages
- **Meaningful inquiries**: Use LLM to detect any important, insightful customer's inquiries about products.

**LOAD DATA**

In [3]:
customer_messages = load_json('../data/customer_messages.json')
message_list = [m['message'] for m in customer_messages]
# load small sample to test
SAMPLE_SIZE = 50
small_sample = message_list[:SAMPLE_SIZE]

**LOAD LLM**

We will use ***Gemini-1.5-flash*** of Google, which is one of the state of the art LLMs (or even Multimodal model) in the present. Furthermore, this model is also provided a good API capacity for free tier.

Because of requirement of precision and static output, we also need to modify `temperature`, `top_p`, and `top_k` to ensure model work accurately.

In [4]:
llm = GoogleGenerativeAI(model='gemini-1.5-flash', temperature=0.1, top_k=10, top_p=0.1)

In [5]:
def parse_output(s: str) -> list:
    start = s.index('[')
    end =  -s[::-1].index(']')
    try:
        res = json.loads(s[start:end])
    except Exception:
        raise Exception(f"Could not parse output. Expected output in either '```python' or '```json' format. Received: {s}")
    
    return res

### Extracting keywords

In [6]:
extract_keywords_chain = extract_keyword_prompt | llm

In [7]:
keywords = extract_keywords_chain.invoke({'input': str(small_sample)})
keywords = parse_output(keywords)
for mess, keyword_list in zip(small_sample, keywords):
    print(f'{mess} -> {keyword_list}')

Em xin menu súp bên mình với ạ -> ['súp', 'menu']
xin menu -> ['menu']
XEM MENU -> ['menu']
XIN MENU -> ['menu']
e xin menu ạ -> ['menu']
XIN MENU -> ['menu']
Em xin menu ạ -> ['menu']
Loại tiểu bảo gồm có thành phần gì ạ -> ['tiểu bảo', 'thành phần']
Bình long tân phú nhận kịp 4h30 ko ạ -> ['bình long tân phú', 'vận chuyển']
196k pk ạ -> ['giá cả']
Kịp ko ạ -> ['vận chuyển']
Tại em làm ở cty 4h30 em ra ca về rồi ạ -> ['vận chuyển']
Vậy canh sao giao tới đây 4h30 dùm em nha -> ['vận chuyển']
Tại 4h30 em mới ra ca -> ['vận chuyển']
32 bình long quận tân phú
0377428748 -> ['bình long', 'tân phú']
Ko ạ -> ['vận chuyển']
Ship sao tới đây 4h 30 dùm em nha -> ['vận chuyển']
Sớm quá em ko nhận đc -> ['vận chuyển']
Đơn hàng của em đi chưa ạ -> ['đơn hàng']
Ok shop -> []
Không ạ , em rất hài lòng về món ăn và cách đóng gói ạ 🥰 -> []
Vâng ạ em nhận đủ -> []
Vâng ạ lần sau em sẽ ủng hộ tiếp món khác ạ -> []
thành phần gồm những gì? -> ['thành phần']
xin menu -> ['menu']
Hi -> []
Cho chị xem menu 

### Important Inquiries

In [8]:
important_inquiry_chain = important_inquiry_prompt | llm

In [9]:
important_message_mask = important_inquiry_chain.invoke({'input': str(small_sample)})

In [10]:
important_message_mask = parse_output(important_message_mask)
for mess, flag in zip(small_sample, important_message_mask):
    print(f'{mess} -> {flag}')

Em xin menu súp bên mình với ạ -> False
xin menu -> False
XEM MENU -> False
XIN MENU -> False
e xin menu ạ -> False
XIN MENU -> False
Em xin menu ạ -> False
Loại tiểu bảo gồm có thành phần gì ạ -> True
Bình long tân phú nhận kịp 4h30 ko ạ -> False
196k pk ạ -> False
Kịp ko ạ -> False
Tại em làm ở cty 4h30 em ra ca về rồi ạ -> False
Vậy canh sao giao tới đây 4h30 dùm em nha -> False
Tại 4h30 em mới ra ca -> False
32 bình long quận tân phú
0377428748 -> False
Ko ạ -> False
Ship sao tới đây 4h 30 dùm em nha -> False
Sớm quá em ko nhận đc -> False
Đơn hàng của em đi chưa ạ -> False
Ok shop -> False
Không ạ , em rất hài lòng về món ăn và cách đóng gói ạ 🥰 -> True
Vâng ạ em nhận đủ -> True
Vâng ạ lần sau em sẽ ủng hộ tiếp món khác ạ -> True
thành phần gồm những gì? -> False
xin menu -> False
Hi -> False
Cho chị xem menu -> False
Ship chị 1 phần 290k nha -> False
Bệnh viện hoàng anh gia lai -> False
Ship lên tầng 5 nha -> False
Tầng 5 bệnh viện hoàng anh gia lai -> False
0905079060 -> False
K